In [2]:
import pandas as pd 

In [3]:
test_wallet_timeseries = pd.read_csv('../data/test_acc_timeseries.csv')

In [4]:
test_wallet_timeseries

total_tx = len(test_wallet_timeseries['signature'].unique())
total_tx

493

In [5]:
test_wallet_timeseries['direction'].unique()

array(['received', 'sent', nan], dtype=object)

In [6]:
received_tx = test_wallet_timeseries[test_wallet_timeseries['direction']=='received']
sent_tx = test_wallet_timeseries[test_wallet_timeseries['direction']=='sent']

In [59]:
sent_tx

,timestamp,signature,net_token_flows,counterparties,type,source,tx_status,direction,counterparty,slot,tx_fee,program_id,PRE_BALANCE,BALANCE,SYMBOL,NAME,MINT,program_name,counterparty_name
6,1743896210,65mbqvyfo9kgav1zdctujdytvzkrb82m2e9zqeddwwdkdi...,"{'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmK...",['752fr9YvQc7gXp1R3C5FwEqWErxTs8uu7AzqV2n9UhAD...,TRANSFER,SOLANA_PROGRAM_LIBRARY,success,sent,NaN,331556730,0.000080,TokenzQdBNbLqP5VEhdkAS6EPFLC1PHnBqCXEpPxuEb,128.319372,0.000000,PYUSD,PayPal USD,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,Unknown Program,Unknown Address
7,1743896146,4ojdpv6epm9zyh5ifuguskt6qe3mndkhlwu7sbnbwpry8c...,{'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB...,['39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5...,UNKNOWN,JUPITER,success,sent,NaN,331556566,0.000080,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,129.499459,0.000000,USDT,Tether,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,Jupiter Swap V6,Unknown Address
11,1743437570,2a7hmpnmcq3qi9xzjfx7xh5w9uascawnvhmbn4cadek586...,"{'SOL': -0.005, 'So111111111111111111111111111...",['HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF...,UNKNOWN,RAYDIUM,success,sent,NaN,330412713,0.000080,DF1ow3DqMj3HvTj8i8J9yM2hE9hCrLLXpdbaKZu4ZPnz,0.025195,0.020115,SOL,Solana,So11111111111111111111111111111111111111111,Unknown Program,Unknown Address
17,1743110889,u6mdfzdapqpszrtwjxh99jtxfzfpxcgc4pjx9spqarhpiq...,"{'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmK...",['DkNrWSTiGoqU2JigKcLu4J7wVhw18AhZ2MVCVaSdHNtS...,TRANSFER,SOLANA_PROGRAM_LIBRARY,success,sent,NaN,329596527,0.000080,TokenzQdBNbLqP5VEhdkAS6EPFLC1PHnBqCXEpPxuEb,493.070747,0.000000,PYUSD,PayPal USD,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,Unknown Program,Unknown Address
20,1743110803,21jcnmgnjlaub17be9ut1jglfcftbdfhkgrguxn1ey9yi1...,{'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB...,['2w4A1eGyjRutakyFdmVyBiLPf98qKxNTC2LpuwhaCruZ...,UNKNOWN,JUPITER,success,sent,NaN,329596313,0.000080,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,497.448706,0.000000,USDT,Tether,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,Jupiter Swap V6,Unknown Address
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,1700437623,3vwe2hdpc5vypneydctmbj77fcfzrgnrzeufnqugczsfvq...,{'So11111111111111111111111111111111111111112'...,['5Q544fKrFoe6tsEbD7S8EmxGTJYAKtTVhAW5Q5pge4j1...,SWAP,JUPITER,success,sent,NaN,231055772,0.000016,JUP2jxvXaqu7NQY1GmNF4m1vodw12LVXYxbFL2uJvfo,0.500000,0.000000,SOL,Wrapped Solana,So11111111111111111111111111111111111111112,Jupiter Swap V2,Unknown Address
484,1700437622,2nfuywjldu3q27v2vxg5swwuklhytw5zqndrxlx7nwfjcn...,{'SOL': -0.50407856},['HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF...,TRANSFER,SYSTEM_PROGRAM,success,sent,NaN,231055769,0.000016,11111111111111111111111111111111,1.565688,1.061593,SOL,Solana,So11111111111111111111111111111111111111111,Unknown Program,Unknown Address
486,1700001026,2c4tsa8zfm19hdxsrqsovr1je22bevcthu3ygjfhxremle...,{'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v...,['82dGS7Jt4Km8ZgwZVRsJ2V6vPXEhVdgDaMP7cqPGG1TW'],UNKNOWN,UNKNOWN,success,sent,82dGS7Jt4Km8ZgwZVRsJ2V6vPXEhVdgDaMP7cqPGG1TW,230045651,0.000007,3parcLrT7WnXAcyPfkCz49oofuuf2guUKkjuFkAhZW8Y,0.225694,0.225688,SOL,Solana,So11111111111111111111111111111111111111111,Parcl V3,Unknown Address
487,1700001006,wclqrautt1gyewwsffuq3sdamvqual8ezbnhczqftkujzv...,{'SOL': -0.00718272},['9jaHQfFgxqFPDv57BRwR24XCCqq3jNp72bGE9597kRFZ'],UNKNOWN,UNKNOWN,success,sent,9jaHQfFgxqFPDv57BRwR24XCCqq3jNp72bGE9597kRFZ,230045602,0.000007,3parcLrT7WnXAcyPfkCz49oofuuf2guUKkjuFkAhZW8Y,0.232884,0.225694,SOL,Solana,So11111111111111111111111111111111111111111,Parcl V3,Unknown Address


In [19]:
sent_tx['net_token_flows'].dropna().head(5).tolist()


["{'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0}",
 "{'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': 0.0, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0}",
 "{'SOL': -0.005, 'So11111111111111111111111111111111111111112': 0.0, 'J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn': 0.0}",
 "{'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0}",
 "{'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': 0.0, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0}"]

In [21]:
sample = sent_tx['net_token_flows'].dropna().iloc[0]
print("Raw value:", sample)

Raw value: {'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0}


In [23]:
if isinstance(sample, str):
    try:
        parsed = ast.literal_eval(sample)
        print("Parsed dict:", parsed)
    except Exception as e:
        print("Failed to parse:", e)

Parsed dict: {'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0}


In [27]:
import ast

parsed_flows = sent_tx['net_token_flows'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
valid_dicts = [d for d in parsed_flows if isinstance(d, dict) and d]
print(f"✅ {len(valid_dicts)} non-empty dicts found")

if valid_dicts:
    print("Sample dict:", valid_dicts[0])

✅ 136 non-empty dicts found
Sample dict: {'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0}


In [31]:
from collections import Counter
import ast

# Parse strings into dictionaries safely
parsed_flows = sent_tx['net_token_flows'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Sum all values grouped by token address
total_flows = sum((Counter(flow) for flow in parsed_flows if isinstance(flow, dict)), Counter())

# Sort by absolute value descending (optional)
sorted_flows = dict(sorted(total_flows.items(), key=lambda item: abs(item[1]), reverse=True))

sorted_flows


{}

In [33]:
parsed_flows = sent_tx['net_token_flows'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
parsed_flows.head(5).tolist()


[{'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0},
 {'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': 0.0,
  '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0},
 {'SOL': -0.005,
  'So11111111111111111111111111111111111111112': 0.0,
  'J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn': 0.0},
 {'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0},
 {'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': 0.0,
  '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0}]

In [35]:
valid_flows = [d for d in parsed_flows if isinstance(d, dict)]
print(f"✅ Found {len(valid_flows)} dicts")
print("Sample:", valid_flows[0])


✅ Found 136 dicts
Sample: {'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0}


In [39]:
from collections import defaultdict

agg = defaultdict(float)

for d in valid_flows:
    for token, amount in d.items():
        agg[token] += amount

# Optionally sort by absolute value
sorted_flows = dict(sorted(agg.items(), key=lambda x: abs(x[1]), reverse=True))
print(sorted_flows)

{'SOL': -31.675079503999992, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 0.0, 'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': 0.0, 'So11111111111111111111111111111111111111112': 0.0, 'J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn': 0.0, 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v': 0.0, '2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv': 0.0, 'WENWENvqqNya429ubCdR81ZmD69brwQaaBYY6p3LCpk': 0.0, 'bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1': 0.0, '9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY': 0.0, '9uvxPXzVYWe8qb7S4gUBqsvASmRMVWo5GSAcMmdZ1LQr': 0.0, 'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So': 0.0, '7dHbWXmci3dT8UFYWYZweBLXgycu7Y3iL6trKn1Y7ARj': 0.0}


In [41]:
pd.DataFrame([sorted_flows])

,SOL,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,So11111111111111111111111111111111111111112,J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,WENWENvqqNya429ubCdR81ZmD69brwQaaBYY6p3LCpk,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,9uvxPXzVYWe8qb7S4gUBqsvASmRMVWo5GSAcMmdZ1LQr,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,7dHbWXmci3dT8UFYWYZweBLXgycu7Y3iL6trKn1Y7ARj
0,-31.67508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
